In [ ]:
import torch
from tqdm import tqdm
from torch.optim import AdamW
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import DataLoader, Dataset
import pandas as pd

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")
if device == "cuda":
    torch.cuda.empty_cache()

Using device: cpu


# Model Object and Properties
Using hateBERT tokenizer and model, with the torch AdamW optimizer. \


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("GroNLP/hateBERT")
model = AutoModelForSequenceClassification.from_pretrained("GroNLP/hateBERT", num_labels=2)  # assuming binary classification (toxic or not)
model = model.to(device)
optimizer = AdamW(model.parameters(), lr=2e-5)

tokenizer_config.json:   0%|          | 0.00/151 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at GroNLP/hateBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Torch Dataset and Dataloader
Dataset is a way to organize data in a standard way for use with the torch library. \
Dataloader takes a torch Dataset and splits it into batches. This gives more control over how much data is used at once, allowing a human operator to prevent the device being used from running out of memory.

In [6]:
# Torch dataset
from torch.utils.data import Dataset
import pandas as pd

class ToxicCommentsDataset(Dataset):
    def __init__(self, comments, labels, tokenizer, max_len):
        self.comments = comments
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.comments)

    def __getitem__(self, item):
        comment = self.comments[item]
        label = self.labels[item]
        encoding = self.tokenizer.encode_plus(
            comment,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Instantiate Dataset
data = pd.read_csv("train.csv")
comments = data["comment_text"]
labels = data["toxic"]

MAX_LEN = 512
BATCH_SIZE = 8
train_dataset = ToxicCommentsDataset(comments.tolist(), labels.tolist(), tokenizer, MAX_LEN)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)



# Fine-tuning HateBERT
The base-model is not trained on out specific task. When run out-of-the-box you get the error message:
>Some weights of BertForSequenceClassification were not initialized from the model checkpoint at GroNLP/hateBERT and are newly initialized: ['classifier.bias', 'classifier.weight']\
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.

And the results are lackluster:\
texts = ["I love you", "I hate you"]\
Sample 0 - Predicted class: 0, Actual class: 0, Confidence: 0.5738734602928162\
Sample 1 - Predicted class: 0, Actual class: 0, Confidence: 0.5975274443626404

Running a fine-tuning pass on the data should give BERT more confidence while evaluating data from the target dataset (even if it takes hours...).

In [ ]:
# Fine-tuning loop
epochs = 3
for epoch in range(epochs):
    model.train()
    total_loss = 0

    for batch in tqdm(train_dataloader):
        optimizer.zero_grad()

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        # Backward pass
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_dataloader)}")

# Save the model after fine-tuning
model.save_pretrained("fine_tuned_hatebert")
tokenizer.save_pretrained("fine_tuned_hatebert")

  1%|          | 1/125 [01:01<2:07:15, 61.58s/it]

# Testing the Model
Tests on the model before fine-tuning were terrible. The model was not confident, and gave the wrong answer most of the time.\
Sample 0 - Predicted class: 0, Actual class: 0, Confidence: 0.5385640263557434\
Sample 1 - Predicted class: 1, Actual class: 0, Confidence: 0.5326736569404602\
Sample 2 - Predicted class: 1, Actual class: 0, Confidence: 0.5592747330665588\
Sample 3 - Predicted class: 1, Actual class: 0, Confidence: 0.5095422267913818\
Sample 4 - Predicted class: 1, Actual class: 0, Confidence: 0.5857343673706055\
Sample 5 - Predicted class: 1, Actual class: 0, Confidence: 0.5527122616767883\
Sample 6 - Predicted class: 1, Actual class: 1, Confidence: 0.5555534958839417\
Sample 7 - Predicted class: 0, Actual class: 0, Confidence: 0.5089718103408813\
Sample 8 - Predicted class: 1, Actual class: 0, Confidence: 0.5100584626197815\


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("GroNLP/hateBERT")

# this will change when I get the good model trained
model = AutoModelForSequenceClassification.from_pretrained("GroNLP/hateBERT")
model = model.to(device) # in case GPU is availabale

# Takes WAAAAAAAAY too long on whole dataset. And my runtime randomly disconnected
# after 25 mins...AND AGAIN AFTER 3HRS OF FINE_TUNING.
texts = comments.iloc[:20].tolist()
# print(texts)
# texts = ["I love you", "I hate you"]

inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=512)
inputs = {k: v.to(device) for k, v in inputs.items()} # make sure input is on the same device as model
#   texts = list of strings to process and run the model on
#   return_tensors = format of output (pt for pytorch, which is what the model needs)
#   padding = whether to pad inputs to the same length
#   truncation = how to treat sequences longer than max length
#   max_length = limits the number of tokens for each input

# This performs a no-training run of the model. Performs terribly with an error
# message saying that it is probably going to perform terribly.
with torch.no_grad():
    outputs = model(**inputs)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at GroNLP/hateBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:

# logits are raw, unnormalized scores output by neural networks
logits = outputs.logits
# If softmax it applied, it makes sense of the otherwise abstract logits
# the resulting probabilities will sum to 1
# it is more for human interpretability (eg. getting the confidence score)
probabilities = torch.softmax(logits, dim=1)

# predicted_class = torch.argmax(probabilities, dim=1)[0].item()
# confidence = probabilities[0][predicted_class].item()
for i in range(len(probabilities)):
    predicted_class = torch.argmax(probabilities[i], dim=0).item()
    confidence = probabilities[i][predicted_class].item()
    print(f"Sample {i} - Predicted class: {predicted_class}, Actual class: {labels[i]}, Confidence: {confidence}")

label_names = ["non-toxic", "toxic"]
print()
# print(f"Prediction: {label_names[predicted_class]} (confidence: {confidence:.4f})")

Sample 0 - Predicted class: 0, Actual class: 0, Confidence: 0.5385640263557434
Sample 1 - Predicted class: 1, Actual class: 0, Confidence: 0.5326736569404602
Sample 2 - Predicted class: 1, Actual class: 0, Confidence: 0.5592747330665588
Sample 3 - Predicted class: 1, Actual class: 0, Confidence: 0.5095422267913818
Sample 4 - Predicted class: 1, Actual class: 0, Confidence: 0.5857343673706055
Sample 5 - Predicted class: 1, Actual class: 0, Confidence: 0.5527122616767883
Sample 6 - Predicted class: 1, Actual class: 1, Confidence: 0.5555534958839417
Sample 7 - Predicted class: 0, Actual class: 0, Confidence: 0.5089718103408813
Sample 8 - Predicted class: 1, Actual class: 0, Confidence: 0.5100584626197815
Sample 9 - Predicted class: 1, Actual class: 0, Confidence: 0.5533177852630615
Sample 10 - Predicted class: 0, Actual class: 0, Confidence: 0.5237893462181091
Sample 11 - Predicted class: 0, Actual class: 0, Confidence: 0.5028075575828552
Sample 12 - Predicted class: 1, Actual class: 1, C